In [1]:
import pandas as pd
from utils.config import domainConfigClass
from typing import List
from collections import Counter
from datetime import datetime


def generateInvoiceId(date: datetime, locationCode: str, invoiceVersion: int):
    return f'{date.strftime("%d%m%Y")}U{locationCode}{invoiceVersion}'


def generatePONo(date: datetime, storeId: str, supplierId: str):
    return f'{date.strftime("%Y%m%d")}-{storeId}-{supplierId}'


def nameExtracter(rightNamesList: List[str], wrongName: str) -> str:
    chances = [0] * len(rightNamesList)

    counterOriginal = Counter(wrongName.lower())
    for i, rightName in enumerate(rightNamesList):
        counterLocation = Counter(rightName.lower())
        chances[i] = (
            sum((counterOriginal & counterLocation).values())
            * 2
            / (sum(counterOriginal.values()) + sum(counterLocation.values()))
        )
    return rightNamesList[chances.index(max(chances))]

In [2]:
df = pd.read_excel("file.xlsx", sheet_name="Sheet1")

In [3]:
dropedDf = df.dropna(how="all")[
    [
        column.rawSheet.columnName
        for column in domainConfigClass.columns
        if column.rawSheet is not None
    ]
]

In [4]:
dropedDf.columns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is not None
]

In [ ]:
pdfColumns = list(
    map(
        lambda x: x.columnName,
        sorted(
            [
                column.invoicePdf
                for column in domainConfigClass.columns
                if column.invoicePdf is not None and column.invoicePdf.index is not None
            ],
            key=lambda x: x.index,
        ),
    )
)

extraColumns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is None and column.invoicePdf is not None
]

date = datetime.strptime(str(df["Date"][0]), "%Y-%m-%d 00:00:00")
invoiceVersion = 1

pdfDF = dropedDf.copy()

for column in extraColumns:
    pdfDF[column] = ""

# pdfDF = pdfDF[pdfColumns]

pdfDF["Dispatched Qty"] = pdfDF["Dispatched Qty"].astype(int)
pdfDF["Rate"] = pdfDF["Rate"].astype(int)

pdfDF["Total Amount"] = pdfDF["Dispatched Qty"] * pdfDF["Rate"]

pdfDF["Location"] = pdfDF["Location"].apply(
    lambda x: nameExtracter(
        [location.locationName for location in domainConfigClass.locations], x
    )
)

In [ ]:
tableDf = {
    location.locationName: {
        "data-frame": pdfDF[pdfDF["Location"] == location.locationName][
            pdfColumns
        ].reset_index(drop=True),
        "invoice-number": generateInvoiceId(date, location.code, invoiceVersion),
        "po-no": generatePONo(date, location.storeId, domainConfigClass.supplierId),
        "shipping-address": location.shippingAddress,
        "retailer": location.retailer,
    }
    for location in domainConfigClass.locations
}

In [ ]:
stringDate = date.strftime("%d-%m-%Y")
invoiceVersion = 1
with pd.ExcelWriter(f"./{stringDate}.xlsx", engine="xlsxwriter") as xlW:
    for loaction in domainConfigClass.locations:
        currentDf = tableDf[loaction.locationName]["data-frame"]
        currentDf["Sr"] = range(1, len(currentDf) + 1)

        sheetName = f'{stringDate} {loaction.locationName}{" " + invoiceVersion if invoiceVersion > 1 else ""}'

        currentDf.to_excel(
            xlW, sheet_name=sheetName, startrow=5, index=False, freeze_panes=(6, 0)
        )

        workBook = xlW.book
        workSheet = xlW.sheets[sheetName]

        headerCellFormat = workBook.add_format(
            {
                "bold": True,
                "align": "center",
                "valign": "vcenter",
                "text_wrap": True,
            }
        )

        bodyCellFormat = workBook.add_format(
            {
                "align": "center",
                "valign": "vcenter",
                "text_wrap": True,
            }
        )

        borderedCellFormat = workBook.add_format(
            {
                "align": "center",
                "valign": "vcenter",
                "text_wrap": True,
                "border": 1,
            }
        )

        # Merge cells for header information
        workSheet.merge_range(
            0, 0, 0, 2, tableDf[loaction.locationName]["retailer"], headerCellFormat
        )
        workSheet.merge_range(1, 0, 1, 2, loaction.locationName, headerCellFormat)
        workSheet.merge_range(
            2,
            0,
            4,
            2,
            f'Shipping Address: {tableDf[loaction.locationName]["shipping-address"]}',
            headerCellFormat,
        )

        workSheet.merge_range(
            0, 3, 0, 7, "Vendor Name: Upgrade Mandi", headerCellFormat
        )
        workSheet.merge_range(1, 3, 1, 7, f"Date: {stringDate}", headerCellFormat)
        workSheet.merge_range(
            2,
            3,
            2,
            7,
            f'Invoice: {tableDf[loaction.locationName]["invoice-number"]}',
            headerCellFormat,
        )
        workSheet.merge_range(
            3, 3, 3, 7, "Email: ankushmisal7387@gmail.com", headerCellFormat
        )
        workSheet.merge_range(
            4,
            3,
            4,
            7,
            f'PO No: {tableDf[loaction.locationName]["po-no"]}',
            headerCellFormat,
        )

        # Set column widths and formats
        workSheet.set_column("A:A", 5, bodyCellFormat)
        workSheet.set_column("B:F", 20, bodyCellFormat)
        workSheet.set_column("G:G", 10, bodyCellFormat)
        workSheet.set_column("H:H", 20, bodyCellFormat)

        workSheet.write(
            6 + len(currentDf),
            4,
            currentDf["Dispatched Qty"].sum(),
            borderedCellFormat,
        )
        workSheet.write(
            6 + len(currentDf),
            7,
            currentDf["Total Amount"].sum(),
            borderedCellFormat,
        )